In [13]:
using LinearAlgebra, JuMP, GLPK
x = zeros(Int32, 3,7)
println(x)
for i=1:3
    for j=1:7
        x[i,j] = 1
    end
end
println(x)
x

Int32[0 0 0 0 0 0 0; 0 0 0 0 0 0 0; 0 0 0 0 0 0 0]
Int32[1 1 1 1 1 1 1; 1 1 1 1 1 1 1; 1 1 1 1 1 1 1]


3×7 Array{Int32,2}:
 1  1  1  1  1  1  1
 1  1  1  1  1  1  1
 1  1  1  1  1  1  1

In [16]:
p=[503 140 203; 675 100 45; 630 105 40; 330 40 295; 105 460 120; 490 55 180; 705 60 400]
p

7×3 Array{Int64,2}:
 503  140  203
 675  100   45
 630  105   40
 330   40  295
 105  460  120
 490   55  180
 705   60  400

In [21]:
using GLPK, JuMP, LinearAlgebra
m = Model(with_optimizer(GLPK.Optimizer))
purchased = [5000 25000 40000; 10000 40000 50000]
market = [1000 4000 8000; 1000 5000 8000]
sqftPerLogW = zeros(2,2,2,3)
sqftPerLogW[:,:,:,1] = cat([400 400;200 200], [200 200;100 100], dims=3)
sqftPerLogW[:,:,:,2] = cat([700 700;500 500], [350 350;250 250], dims=3)
sqftPerLogW[:,:,:,3] = cat([900 900;1300 1300], [450 450;650 650], dims=3)
costPerLog = [340 190; 490 140]
costPerSqft = [1 0.3 0.1; 2.2 0.6 0.2]
price = [45 40 33; 75 65 50]                    # per sheet(1/4, 1/4, Ab, ac, bc)
@variable(m, w[1:2, 1:2, 1:2] >= 0)             # w(q,v,t)(log)
@variable(m, x[1:2, 1:3] >= 0)                  # x(t,g)(sqft)
@variable(m, y[1:2, 1:3, 1:3] >= 0)             # y(t,g,g)(sheets)
@variable(m, z[1:2, 1:3] >= 0)                  # z(t,g,g)(sheets)
@constraint(m, sum(w[1,1,:]) <= 200)    # log avalability
@constraint(m, sum(w[1,2,:]) <= 100)   
@constraint(m, sum(w[2,1,:]) <= 300)
@constraint(m, sum(w[2,2,:]) <= 1000)
for i=1:2
    for j=1:3
        @constraint(m, x[i,j] <= purchased[i,j])# purchased constraint
        @constraint(m, z[i,j] <= market[i,j])   # market constraint
    end
end
@constraint(m, 0.25*sum(z[1,:]) + 0.4*sum(z[2,:]) <= 4500)     #pressing constraint

@constraint(m, sum(sqftPerLogW[:,:,1,1].*w[:,:,1]) + x[1,1] >= 35*sum(y[1,1,1:2]))  # balance constraint
@constraint(m, sum(sqftPerLogW[:,:,1,2].*w[:,:,1]) + x[1,2] >= 35*sum(y[1,2,1:3]))
@constraint(m, sum(sqftPerLogW[:,:,1,3].*w[:,:,1]) + x[1,3] >= 35*sum(y[1,3,2:3]))

@constraint(m, sum(sqftPerLogW[:,:,2,1].*w[:,:,2]) + x[2,1] >= 35*sum(y[2,1,1:2]))
@constraint(m, sum(sqftPerLogW[:,:,2,2].*w[:,:,2]) + x[2,1] >= 35*sum(y[2,2,1:3]))
@constraint(m, sum(sqftPerLogW[:,:,2,3].*w[:,:,2]) + x[2,1] >= 35*sum(y[2,3,2:3]))

@constraint(m, sum(y[1,1:2,1]) == sum(z[:,1:2]))
@constraint(m, sum(y[1,:,2]) == sum(z[:,1]) + sum(z[:,3]))
@constraint(m, sum(y[1,2:3,3]) == sum(z[:,2:3]))

# @constraint(m, sum(y[2,1:2,1]) == sum(z[:,1:2]))
@constraint(m, sum(y[2,:,2]) == sum(z[2,:]))
@constraint(m, sum(y[2,2:3,3]) == sum(z[1,:]) + 2*sum(z[2,:]))

@objective(m, Max, -(sum(costPerLog.*w[:,:,1]) + sum(costPerLog.*w[:,:,2])) - (sum(costPerSqft.*x)) 
           + sum(price.*z))

m

A JuMP Model
Maximization problem with:
Variables: 38
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 38 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 5 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 17 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: w, x, y, z

In [26]:
optimize!(m)
println((objective_value(m)))

438357.1428571427


MethodError: MethodError: no method matching value(::Array{UnitRange{Int64},1})
Closest candidates are:
  value(!Matched::NonlinearExpression) at /home/son/.julia/packages/JuMP/tyMag/src/nlp.jl:1126
  value(!Matched::NonlinearParameter) at /home/son/.julia/packages/JuMP/tyMag/src/nlp.jl:125
  value(!Matched::VariableRef) at /home/son/.julia/packages/JuMP/tyMag/src/variables.jl:721
  ...

In [23]:
x = [1:2, 1:2, 1:4]
y = zeros(Int32, 2,2,4)
# print(y)
y[1,2,2] = 3
# A = [1 2; 3 4]
A = zeros(2,2,2)
A[:,:,1] = [111 121; 211 221]
A[:,:,2] = [10 20; 30 40]
println(A[:,:,1])
z = [1:2, 1:2, 1:4]
b = zeros(2,2,2,3)
b[:,:,:,1] = cat([400 400;200 200], [200 200;100 100], dims=3)
b[:,:,:,2] = cat([700 700;500 500], [350 350;250 250], dims=3)
b[:,:,:,3] = cat([900 900;1300 1300], [450 450;650 650], dims=3)
# println((A[:,:,1].*b[1,:,:,1]))
# println(b[1,:,:,1])
# println(A[:,:,1])
# print(cat(A[:,:,1], dims=1))
# println(b[1,1,1,1])
# println(b[1,1,2,1])
# println(b[1,2,1,1])
# println(b[1,2,2,1])
# println(b[2,1,1,1])
# println(b[2,1,2,1])
# println(b[2,2,1,1])
# println(b[2,2,2,1])
costPerLog = [340 190; 490 140]
println(costPerLog[1,1])
println(costPerLog[1,2])
println(costPerLog[2,1])
println(costPerLog[2,2])
println(costPerLog.*A[:,:,1])

[111.0 121.0; 211.0 221.0]
340
190
490
140
[37740.0 22990.0; 103390.0 30940.0]
